In [1]:
import sys, os
%matplotlib qt
import pydip

In [1]:
# This scripts contains an example of how to use PyDip.
from pprint import pprint

from pydip.map.predefined import vanilla_dip
from pydip.player import Player, command
from pydip.turn import resolve_turn, resolve_retreats, resolve_adjustment

# The test module contains some convenient helpers (not used in this examples,
# in order to better demonstrate how PyDip works)
from pydip.test import CommandHelper, PlayerHelper, TurnHelper, CommandType


In [8]:


# PyDip provides a predefined map of vanilla Diplomacy
# the most extensive generator defines the map, supply centers and ownership
vanilla_map = vanilla_dip.generate_starting_ownership_map()

game_map = vanilla_map.supply_map.game_map

In [11]:
# vanilla_dip includes a definition that can be used to create players
print('\nPlayers\n=======\n')
players = {}
for name, units in vanilla_dip.generate_starting_player_units().items():
    starting_config = [dict(territory_name=u.position, unit_type=u.unit_type) for u in units]
    players[name] = Player(name, game_map, starting_config)
    print(players[name])


# players can issue commands
england = players['England']
unit = england.find_unit('Liverpool')
cmd = command.MoveCommand(england, unit, 'Wales')
print('\nCommands\n========\n{}'.format(cmd))
# illegal commands result in an error
try:
    command.MoveCommand(england, unit, 'York')
except AssertionError:
    print('Illegal command\n')


Players

England: {'London', 'Edinburgh', 'Liverpool'}
  Units: [[FLEET -- Edinburgh Coast], [TROOP -- Liverpool], [FLEET -- London Coast]]
France: {'Marseilles', 'Paris', 'Brest'}
  Units: [[TROOP -- Marseilles], [FLEET -- Brest Coast], [TROOP -- Paris]]
Germany: {'Kiel', 'Berlin', 'Munich'}
  Units: [[FLEET -- Kiel Coast], [TROOP -- Berlin], [TROOP -- Munich]]
Italy: {'Venice', 'Rome', 'Naples'}
  Units: [[TROOP -- Venice], [TROOP -- Rome], [FLEET -- Naples Coast]]
Russia: {'Moscow', 'St. Petersburg', 'Sevastopol', 'Warsaw'}
  Units: [[TROOP -- Warsaw], [TROOP -- Moscow], [FLEET -- St. Petersburg South Coast], [FLEET -- Sevastopol Coast]]
Austria: {'Vienna', 'Budapest', 'Trieste'}
  Units: [[FLEET -- Trieste Coast], [TROOP -- Vienna], [TROOP -- Budapest]]
Turkey: {'Ankara', 'Constantinople', 'Smyrna'}
  Units: [[TROOP -- Constantinople], [TROOP -- Smyrna], [FLEET -- Ankara Coast]]

Commands
England: TROOP Liverpool -> Wales
Illegal command



In [12]:
# PyDip can resolve a set of commands
# the pydip test package provides some helpers
commands = [cmd]
for name, moves in dict(
        England=[('Edinburgh Coast', 'Norwegian Sea'),
                 ('Liverpool', 'Wales'),
                 ('London Coast', 'North Sea')],
        France=[('Brest Coast', 'Mid-Atlantic Ocean'),
                ('Marseilles', 'Burgundy'),
                ('Paris', 'Picardy')],
        Germany=[('Berlin', 'Prussia'),
                 ('Kiel Coast', 'Holland Coast'),
                 ('Munich', 'Burgundy')],
        Italy=[('Naples Coast', 'Ionian Sea'),
               ('Rome', 'Naples'),
               ('Venice', 'Tyrolia')],
        Russia=[('Moscow', 'Livonia'),
                ('Sevastopol Coast', 'Black Sea'),
                ('St. Petersburg South Coast', 'Livonia Coast'),
                ('Warsaw', 'Ukraine')],
        Austria=[('Budapest', 'Serbia'),
                 ('Trieste Coast', 'Albania Coast'),
                 ('Vienna', 'Tyrolia')],
        Turkey=[('Ankara Coast', 'Black Sea'),
                ('Constantinople', 'Bulgaria'),
                ('Smyrna', 'Constantinople')],
).items():
    player = players[name]
    for position, dest in moves:
        unit = player.find_unit(position)
        cmd = command.MoveCommand(player, unit, dest)
        print(cmd)
        commands.append(cmd)

# resolve_turn returns a map of units (at their new positions) that tells
# if and where units must retreat
resolutions = resolve_turn(game_map, commands)
print('\nResolution\n==========')
pprint(resolutions)

England: FLEET Edinburgh Coast -> Norwegian Sea
England: TROOP Liverpool -> Wales
England: FLEET London Coast -> North Sea
France: FLEET Brest Coast -> Mid-Atlantic Ocean
France: TROOP Marseilles -> Burgundy
France: TROOP Paris -> Picardy
Germany: TROOP Berlin -> Prussia
Germany: FLEET Kiel Coast -> Holland Coast
Germany: TROOP Munich -> Burgundy
Italy: FLEET Naples Coast -> Ionian Sea
Italy: TROOP Rome -> Naples
Italy: TROOP Venice -> Tyrolia
Russia: TROOP Moscow -> Livonia
Russia: FLEET Sevastopol Coast -> Black Sea
Russia: FLEET St. Petersburg South Coast -> Livonia Coast
Russia: TROOP Warsaw -> Ukraine
Austria: TROOP Budapest -> Serbia
Austria: FLEET Trieste Coast -> Albania Coast
Austria: TROOP Vienna -> Tyrolia
Turkey: FLEET Ankara Coast -> Black Sea
Turkey: TROOP Constantinople -> Bulgaria
Turkey: TROOP Smyrna -> Constantinople

Resolution
defaultdict(<class 'dict'>,
            {'Austria': {[FLEET -- Albania Coast]: None,
                         [TROOP -- Serbia]: None,
      

In [2]:
import matplotlib.pyplot as plt


class LineDrawer(object):
    lines = []
    def draw_line(self):
        ax = plt.gca()
        xy = plt.ginput(2)

        x = [p[0] for p in xy]
        y = [p[1] for p in xy]
        line = plt.plot(x,y)
        ax.figure.canvas.draw()

        self.lines.append(line)


def onclick(event):
    if event.dblclick:
        if event.button == 1:
            # Draw line
            ld = LineDrawer()
            ld.draw_line() # here you click on the plot
        elif event.button == 3:
            # Write to figure
            plt.figtext(3, 8, 'boxed italics text in data coords', style='italic', bbox={'facecolor':'red', 'alpha':0.5, 'pad':10})
            circ = plt.Circle((event.x, event.y), radius=0.07, color='g')
            ax.add_patch(circ)
            plt.draw()
        else:
            pass # Do nothing


def onpick(event):
    thisline = event.artist
    xdata = thisline.get_xdata()
    ydata = thisline.get_ydata()
    ind = event.ind
    print ('onpick points:', zip(xdata[ind], ydata[ind]))



fig, ax = plt.subplots()

connection_id = fig.canvas.mpl_connect('button_press_event', onclick)
fig.canvas.mpl_connect('pick_event', onpick)


plt.tight_layout()

plt.show()

In [1]:
import sys, os
import numpy as np
from omnibelt import load_yaml, save_yaml
%matplotlib qt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
root = '/mnt/c/Users/anwan/Nextcloud/Khan/projects/ludos/examples/diplomacy'
root = r'C:\Users\anwan\Nextcloud\Khan\projects\ludos\examples\diplomacy'
image_path = os.path.join(root, 'extra', 'vanilla_blank.gif')
edge_path = os.path.join(root, 'data', 'base_edges.yaml')
node_path = os.path.join(root, 'data', 'base_nodes.yaml')
pos_path = os.path.join(root, 'data', 'pos.yaml')

In [3]:
edges = load_yaml(edge_path)
nodes = load_yaml(node_path)

In [4]:
nodes

{'nao': {'name': 'North Atlantic Ocean', 'type': 'sea'},
 'mao': {'name': 'Mid Atlantic Ocean', 'type': 'sea'},
 'por': {'name': 'Portugal', 'type': 'coast'},
 'naf': {'name': 'North Africa', 'type': 'coast'},
 'iri': {'name': 'Irish Sea', 'type': 'sea'},
 'eng': {'name': 'English Channel', 'type': 'sea'},
 'bre': {'name': 'Brest', 'type': 'coast'},
 'gas': {'name': 'Gascony', 'type': 'coast'},
 'spa': {'name': 'Spain', 'type': 'coast', 'coasts': ['nc', 'sc']},
 'wes': {'name': 'Western Meditteranean Sea', 'type': 'sea'},
 'tun': {'name': 'Tunisia', 'type': 'coast', 'sc': 1},
 'nwg': {'name': 'Norwegian Sea', 'type': 'sea'},
 'nth': {'name': 'Nothern Sea', 'type': 'sea'},
 'cly': {'name': 'Clyde', 'type': 'coast'},
 'lvp': {'name': 'Liverpool', 'type': 'coast', 'sc': 1},
 'wal': {'name': 'Wales', 'type': 'coast'},
 'edi': {'name': 'Edinburgh', 'type': 'coast', 'sc': 1},
 'yor': {'name': 'York', 'type': 'coast'},
 'lon': {'name': 'London', 'type': 'coast', 'sc': 1},
 'pic': {'name': 'Pi

In [5]:
coasts = {}

for ID, node in nodes.items():
    if 'coasts' in node:
        name = node['name']
        for c in node['coasts']:
            coasts[f'{ID}-{c}'] = {'name': f'{name} ({c.upper()})', 'type': 'coast', 'coast-of': ID}

nodes.update(coasts)

In [6]:
base_pos = True
edge_pos = True
symmetric_edges = True
coast_retreat_pos = False
retreat_pos = True

In [7]:
try:
    pos = load_yaml(pos_path)
    print(f'Found previous pos file with {len(pos)} entries')
except FileNotFoundError:
    pos = {}
    print('No pos file found, starting from scratch')

No pos file found, starting from scratch


In [8]:
fig, ax = plt.subplots(figsize=(12,8))


In [9]:

img = mpimg.imread(image_path)

plt.imshow(img)

todo = list(nodes.keys())
done = []
key = None
current = todo.pop()
target = None

def _post_process(): # make sure coastal tiles also have a retreat pos
    
    for ID, p in pos.items():
        if retreat_pos and 'retreat' not in p:
            
            assert ID in nodes, f'unknown {ID}'
            if 'coast-of' in nodes[ID]:
                base = nodes[ID]['coast-of']
                assert 'retreat' in pos[base], f'unknown: {base}'
                p['retreat'] = pos[base]['retreat']
        
    

def _next_prompt():
    global current, key, target, todo, done
    
    if current is None:
        current = todo.pop()
    
    if current not in pos:
        pos[current] = {}
    
    name = current.upper() if nodes is None or current not in nodes else nodes[current]['name']
    
    while len(todo):
        
        if current is None:
            current = todo.pop()
    
        if current not in pos:
            pos[current] = {}
        
        mypos = pos[current]
        
        if edge_pos and 'edges' not in mypos:
            pos[current]['edges'] = {}
        
        
        if base_pos and 'base' not in mypos:
            plt.title(f'{name} - base')
            key = 'base'
            target = None
            return

        elif 'coast' not in nodes[current] and retreat_pos and 'retreat' not in mypos:
            plt.title(f'{name} - retreat')
            key = 'retreat'
            target = None
            return

        elif 'coast' in nodes[current] and coast_retreat_pos and 'retreat' not in mypos:
            plt.title(f'{name} - retreat')
            key = 'retreat'
            target = None
            return
            
        elif edge_pos and len(edges[current]) != len(mypos['edges']):
            key = 'edges'
            target = None
            for edge in edges[current]:
                if edge not in mypos['edges']:
                    target = edge
            
            plt.title(f'{name} - edge with {target}')
            
            if key not in pos[current]:
                pos[current][key] = {}
            if target not in pos:
                pos[target] = {}
            if key not in pos[target]:
                pos[target][key] = {}

        else: # current is done
            done.append(current)
            print('Done with {current}')
            current = None
                
                
    # no more todo
    
    fig.canvas.mpl_disconnect(cid)

    plt.title('Done!')

    _post_process()

#         save_yaml(pos, pos_path)
    print(f'All positions collected, saved pos to: {pos_path}')
#             fig.close()
#             quit()
    raise Exception('done')
                
    pass

def _prev_node():
    
    global current
    
    if current is None:
        pass
    elif len(pos[current]):
        
        pos[current].clear()
        
        current = None
        
    elif len(done):
        todo.append(current)
        
        old = done.pop()
        pos[old].clear()
        todo.append(old)
        
        
        current = None
        
    _next_prompt()
    

def onclick(event):
    btn = event.button # 1 is left, 3 is right
    xy = [event.xdata, event.ydata]
    
    if btn == 1:
        print(f'Saving {current} {key} {target}: {xy}')
    
        if target is None:
            pos[current][key] = xy
        else:
            pos[current][key][target] = xy
            if symmetric_edges:
                pos[target][key][current] = xy

        _next_prompt()
    
    elif btn == 3:
        print('Going back')
        _prev_node()
    
cid = fig.canvas.mpl_connect('button_press_event', onclick)

_next_prompt()

Saving bul-sc base None: [751.3558509920366, 806.3659738156296]


In [ ]:
# save_yaml(pos, pos_path)

In [10]:
pos

{'bul-sc': {'edges': {}, 'base': [751.3558509920366, 806.3659738156296]}}